This notebook will:
- Download ZTF alerts for a named SN from the DESY archive.
- Load a set of AMPEL feature extraction and ML classification units.
- Apply these to each phase of the transient, using an assumbed redshift.
- Plot the evolution of SNIa probability provided by each classifier.

In [ ]:
# Name (ZTF ID) of transient and redshift to use (when relevant)

# Objects from ZTF experiment
snname, z_assumed = "ZTF24abiaxkd", 0.07  # SNII
#snname, z_assumed = "ZTF24abiesnr", 0.07  # SNIbn
#snname, z_assumed = "ZTF24abikstg", 0.07  # SNIc





In [ ]:
import os
import requests
import warnings
import yaml
warnings.filterwarnings('ignore')

# Plotting
import matplotlib.pyplot as plt
from IPython.core.display import Image, display
import seaborn as sns
import numpy as np

# Base AMPEL tools
from ampel.log.AmpelLogger import AmpelLogger
from ampel.content.T1Document import T1Document 
from ampel.dev.DevAmpelContext import DevAmpelContext
from ampel_notebook_utils import api_get_datapoints

# Units linking to feature extraction and ML algorithms
from ampel.contrib.hu.t2.T2RunParsnipRiseDecline import T2RunParsnipRiseDecline
from ampel.contrib.hu.t2.T2RunParsnipRiseDecline import get_probability_evolution

In [ ]:
# Path to AMPEL configuration file
AMPEL_CONF = '../ampel_conf.yaml'
# Personal access token to the DESY ZTF archive
token = os.environ["ARCHIVE_TOKEN"]

In [ ]:
# Parse the system for paths to analysis units
ctx = DevAmpelContext.load(
    config = AMPEL_CONF,
    db_prefix = 'dumpme',
    purge_db = True,
)

### Obtain data from archive

In [ ]:
dps = api_get_datapoints(snname, token)
print(f'Found {len(dps)} datapoints')

In [ ]:
# Make sure datapoints sorted according to increasing time 
dps = sorted(dps, key=lambda x: x['body']['jd'])

In [ ]:
# Record detection date for plotting
jddet = min([dp['body']['jd'] for dp in dps if 'magpsf' in dp['body']])
print(f'First detected at JD {jddet}')

### Configure and loading classifiers

In [ ]:
# This configuration file is provided in full
# It lists the detailed descrpiptor of all classifiers to use, including local paths to 
# classifier input files.
rdp_config = { 
   'classifier_name': 'rdp',
   'classifier_version': "0.1",
   't_cadence': 3.,
   'significant_bands': ['ztfg',  'ztfr', 'ztfi'],
   'paths_parsnip': {
       'snlong': {
                'model': '/Users/jnordin/data/models/parsnip/v2_Mar2025/noiztfmodel_sncut_scale3_z3_subsamp0.9_cadscale0.5_seed0.pt',
                'classifier': '/Users/jnordin/data/models/parsnip/v2_Mar2025/noiztf_classifier_sncut_scale3_z3_subsamp0.9_cadscale0.5_seed0.pkl',
       }
    },
    'paths_xgbmulti': {
           # https://box.hu-berlin.de/d/d997be8614934fde8acc/ 
        'sample1': {
            'path': '/Users/jnordin/data/models/xgb/v1_2024/xgb24_sn_long_sample1',
            'classes': [ "slsn", "snia", "snibc", "snii", "sniin" ],
        },
        'later': {
            'path': '/Users/jnordin/data/models/xgb/v1_2024/xgb24_sn_long_latephase',
            'classes': [ "slsn", "snia", "snibc", "snii", "sniin" ],
        },
        'early': {
            'path': '/Users/jnordin/data/models/xgb/v1_2024/xgb24_sn_long_earlyphase',
            'classes': [ "slsn", "snia", "snibc", "snii", "sniin" ],
        },
        'last_parsnip': {
            'path': '/Users/jnordin/data/models/xgb/v1_2024/xgb24_sn_long_lastalert_parsnip',
            'classes': [ "slsn", "snia", "snibc", "snii", "sniin" ],
        },
        'last_risedec': {
            'path': '/Users/jnordin/data/models/xgb/v1_2024/xgb24_sn_long_lastalert_risedecline',
            'classes': [ "slsn", "snia", "snibc", "snii", "sniin" ],
        },
        'last_prd': {
            'path': '/Users/jnordin/data/models/xgb/v1_2024/xgb24_sn_long_lastalert_risedeclineparsnip',
            'classes': [ "slsn", "snia", "snibc", "snii", "sniin" ],
        }
    },
    'paths_xgbbinary': {},
    'parsnip_zeropoint_offset': 0,
    'add_parsnip_from': 'snlong', 
    'parsnipplot_suffix': 'png',
    'parsnipplot_dir': '/Users/jnordin/tmp/parsnipPlot',
    'fixed_z': z_assumed,
    'redshift_kind': None,
    'return_features': True,
    'tabulator': [ {'unit': 'ZTFT2Tabulator'} ],                
    't2_dependency': [],
}

In [ ]:
# This configuration file is provided in full
# It lists the detailed descrpiptor of all classifiers to use, including local paths to 
# classifier input files.
rdp_config = { 
   'classifier_name': 'rdp',
   'classifier_version': "0.1",
   't_cadence': 3.,
   'significant_bands': ['ztfg',  'ztfr', 'ztfi'],
   'paths_parsnip': {
       'snlong': {
                'model': '/Users/jnordin/data/models/parsnip/v2_Mar2025/noiztfmodel_sncut_scale3_z3_subsamp0.9_cadscale0.5_seed0.pt',
                'classifier': '/Users/jnordin/data/models/parsnip/v2_Mar2025/noiztf_classifier_sncut_scale3_z3_subsamp0.9_cadscale0.5_seed0.pkl',
       }
    },
    'paths_xgbmulti': {
    },
    'paths_xgbbinary': {},
    'parsnip_zeropoint_offset': 0,
    'add_parsnip_from': 'snlong', 
    'parsnipplot_suffix': 'png',
    'parsnipplot_dir': '/Users/jnordin/tmp/parsnipPlot',
    'fixed_z': z_assumed,
    'redshift_kind': None,
    'return_features': True,
    'tabulator': [ {'unit': 'ZTFT2Tabulator'} ],                
    't2_dependency': [],
}

In [ ]:
# Model/result labeling corresponding to the above, used while plotting.
classlabeling = {
    'Parsnip': {'targetname': 'SNIa', 'model': 'parsnip', 'training': 'snlong' },
    'XRDSampling': {'targetname': 'snia', 'model': 'xgbmulti', 'training': 'sample1' },    
    'XRDPostpeak': {'targetname': 'snia', 'model': 'xgbmulti', 'training': 'later' },    
    'XRDEarly': {'targetname': 'snia', 'model': 'xgbmulti', 'training': 'early' },    
    'XParsnipLast': {'targetname': 'snia', 'model': 'xgbmulti', 'training': 'last_parsnip' },    
    'XRDLast': {'targetname': 'snia', 'model': 'xgbmulti', 'training': 'last_risedec' },    
    'XRDPLast': {'targetname': 'snia', 'model': 'xgbmulti', 'training': 'last_prd' },    
}
# For the second configuration, only using parsnip
classlabeling = {
    'Parsnip': {'targetname': 'SNIa', 'model': 'parsnip', 'training': 'snlong' },
}


In [ ]:
# Load base classifier unit 
t2unit = T2RunParsnipRiseDecline( **rdp_config, logger=AmpelLogger.get_logger() )

In [ ]:
# Read classifier files 
t2unit.post_init()

### Run classifiers.
Loop through each dp, gradually increase dps range and study evolution of probability. 

In [ ]:
classifier_results = []
for k in range(len(dps)):
    if not 'magpsf' in dps[k]['body']:
        continue
    print("Alert {} @ JD {}".format(k, dps[k]['body']['jd'] ))
    classifier_results.append( t2unit.process(T1Document(stock=dps[0]['stock'], link=0), dps[0:k+1], t2_views=[]) )


### Plot results 

In [ ]:
_ = plt.figure(1,figsize=(12,12))

fig, axs = plt.subplots(2, 1, sharex=True)
fig.subplots_adjust(hspace=0)

plt.subplot(2,1,1)

plt.title( snname )

# g
phot = np.array( [
            (dp['body']['jd']-jddet,dp['body']['magpsf'],dp['body']['sigmapsf']) 
            for dp in dps if 'magpsf' in dp['body'] and dp['body']['fid']==1 
])
if len(phot)>0:
    ax = sns.lineplot(x=phot[:,0],y=phot[:,1], label='ZTF g', color='green')
    ax.errorbar(phot[:,0], phot[:,1], yerr=phot[:,2], fmt='o', color='k', alpha=0.5)

# R
phot = np.array( [
            (dp['body']['jd']-jddet,dp['body']['magpsf'],dp['body']['sigmapsf']) 
            for dp in dps if 'magpsf' in dp['body'] and dp['body']['fid']==2 
])
if len(phot)>0:
    ax = sns.lineplot(x=phot[:,0],y=phot[:,1], label='ZTF R', color='red')
    ax.errorbar(phot[:,0], phot[:,1], yerr=phot[:,2], fmt='o', color='k', alpha=0.5)

# i
phot = np.array( [
            (dp['body']['jd']-jddet,dp['body']['magpsf'],dp['body']['sigmapsf']) 
            for dp in dps if 'magpsf' in dp['body'] and dp['body']['fid']==3 
])
if len(phot)>0:
    ax = sns.lineplot(x=phot[:,0],y=phot[:,1], label='ZTF i', color='green')
    ax.errorbar(phot[:,0], phot[:,1], yerr=phot[:,2], fmt='o', color='k', alpha=0.5)

plt.gca().invert_yaxis()
plt.xlabel('Time (days wrt to JD {}'.format(jddet))
plt.ylabel('Magnitude')


plt.subplot(2,1,2)

colors = sns.color_palette(n_colors=len(classlabeling))
for modellabel, modelkeys in classlabeling.items():
    print(modelkeys['targetname'])
    print(modelkeys['model'])
    print(modelkeys['training'])
    t, c = get_probability_evolution( 
        [t2res.body for t2res in classifier_results], 
        modelkeys['targetname'], modelkeys['model'], "rdp", 
        classlabel=modelkeys['training'] 
    )
    sns.lineplot(x=[ti-jddet for ti in t],y=c, label=modellabel, color=colors.pop(), linewidth=3, alpha=0.7 )

plt.ylabel('{} Probability'.format(
    set([v['targetname'] for k,v in classlabeling.items()]) ) 
          )
plt.xlabel('Time (days wrt to JD {})'.format(jddet))

_ = plt.legend(loc=1, bbox_to_anchor=(1.32, 1.),
          fancybox=False, shadow=False, ncol=1)
plt.show()

In [ ]:
# Above plot shows results for a SNIa ... change the targetname to pick up another class and rerun plot
classlabeling = {
    'Parsnip': {'targetname': 'SNII', 'model': 'parsnip', 'training': 'snlong' },
    'XRDSampling': {'targetname': 'snii', 'model': 'xgbmulti', 'training': 'sample1' },    
    'XRDPostpeak': {'targetname': 'snii', 'model': 'xgbmulti', 'training': 'later' },    
    'XRDEarly': {'targetname': 'snii', 'model': 'xgbmulti', 'training': 'early' },    
    'XParsnipLast': {'targetname': 'snii', 'model': 'xgbmulti', 'training': 'last_parsnip' },    
    'XRDLast': {'targetname': 'snii', 'model': 'xgbmulti', 'training': 'last_risedec' },    
    'XRDPLast': {'targetname': 'snii', 'model': 'xgbmulti', 'training': 'last_prd' },    
}


In [ ]:
# Above plot shows results for a SNIa ... change the targetname to pick up another class and rerun plot
classlabeling = {
    'Parsnip': {'targetname': 'SNIbc', 'model': 'parsnip', 'training': 'snlong' },
    'XRDSampling': {'targetname': 'snibc', 'model': 'xgbmulti', 'training': 'sample1' },    
    'XRDPostpeak': {'targetname': 'snibc', 'model': 'xgbmulti', 'training': 'later' },    
    'XRDEarly': {'targetname': 'snibc', 'model': 'xgbmulti', 'training': 'early' },    
    'XParsnipLast': {'targetname': 'snibc', 'model': 'xgbmulti', 'training': 'last_parsnip' },    
    'XRDLast': {'targetname': 'snibc', 'model': 'xgbmulti', 'training': 'last_risedec' },    
    'XRDPLast': {'targetname': 'snibc', 'model': 'xgbmulti', 'training': 'last_prd' },    
}

In [ ]:
# In case we only ran parsnip
classlabeling = {
    'Parsnip': {'targetname': 'SNII', 'model': 'parsnip', 'training': 'snlong' },
}